In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

import lightgbm as lgb

In [2]:
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical,plot_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

2025-11-24 04:44:34.158832: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763959474.409456      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763959474.480514      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [3]:
def new_feature(df):
  

    # ----------------------------
    # Feature Engineering
    # ----------------------------
    
    # 1. Debt-to-Income Ratio (DTI)
    df['dti'] = df['loan_amount'] / df['annual_income']
    
    # 2. Loan-to-Income Ratio
    df['loan_to_income'] = df['loan_amount'] / df['annual_income']
    
    # 3. Effective Interest Amount
    df['interest_amount'] = df['loan_amount'] * df['interest_rate']
    
    # 4. Interest-to-Income Ratio
    df['interest_to_income'] = df['interest_amount'] / df['annual_income']
    
   
    
    # 7. Risk Score (higher = riskier)
    df['risk_score'] = df['loan_to_income'] * (850 - df['credit_score'])
    
    # 8. Normalized / Log Features
    df['income_log'] = np.log1p(df['annual_income'])
    df['loan_log'] = np.log1p(df['loan_amount'])
    df['interest_log'] = np.log1p(df['interest_amount'])

    return df
    

In [4]:
scaler = StandardScaler()
le1 = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()
le4 = LabelEncoder()
le5 = LabelEncoder()
def preprocessing_train(filepath):
    df = pd.read_csv(filepath)
    
   
    df['grade_subgrade'] = df['grade_subgrade'].str.replace(r'\d+', '', regex=True)
    df['grade_subgrade'] = le1.fit_transform(df['grade_subgrade'])
    df['gender'] = le2.fit_transform(df['gender'])
    df['marital_status'] = le3.fit_transform(df['marital_status'])
    df['education_level'] = le4.fit_transform(df['education_level'])
    df['loan_purpose'] = le5.fit_transform(df['loan_purpose'])
    
    df = pd.get_dummies(df, columns=['employment_status'], drop_first=True)
    df = df.drop(columns=['id'])
    
    # Target
    y = df['loan_paid_back']
    y = to_categorical(y)

    # Features
    
    X = df.drop(columns=['loan_paid_back'])
    X=new_feature(X)
    # Select only specific columns
    cols_to_scale =['annual_income','credit_score','loan_amount','interest_rate','dti','loan_to_income','interest_amount','interest_to_income','risk_score']   # modify here
    
   
    X[cols_to_scale] = scaler.fit_transform(X[cols_to_scale])

    return X, y


In [5]:

def preprocessing_test(filepath):
    df = pd.read_csv(filepath)

    
   
    df['grade_subgrade'] = df['grade_subgrade'].str.replace(r'\d+', '', regex=True)
    df['grade_subgrade'] = le1.fit_transform(df['grade_subgrade'])
    df['gender'] = le2.fit_transform(df['gender'])
    df['marital_status'] = le3.fit_transform(df['marital_status'])
    df['education_level'] = le4.fit_transform(df['education_level'])
    df['loan_purpose'] = le5.fit_transform(df['loan_purpose'])
    
    df = pd.get_dummies(df, columns=['employment_status'], drop_first=True)
    df = df.drop(columns=['id'])
    # Features
    X = df
    X=new_feature(X)
    # Select only specific columns
    cols_to_scale =['annual_income','credit_score','loan_amount','interest_rate','dti','loan_to_income','interest_amount','interest_to_income','risk_score']   # modify here
    X[cols_to_scale] = scaler.transform(X[cols_to_scale])

    return X


In [6]:
X_train,y_train=preprocessing_train('/kaggle/input/playground-series-s5e11/train.csv')
X_test=preprocessing_test('/kaggle/input/playground-series-s5e11/test.csv')

NameError: name 'np' is not defined

In [ ]:
X_test

In [ ]:
X_train

In [ ]:
#pd.crosstab(df['marital_status'],df['loan_paid_back'],normalize='index')

In [ ]:
#Model construction
inputs=Input(shape=(22,))
y=Dense(256,activation='relu')(inputs)
y=Dropout(0.1)(y)

y=Dense(128,activation='relu')(y)
y=Dropout(0.1)(y)
y=Dense(64,activation='relu')(y)
y=Dropout(0.1)(y)
y=Dense(32,activation='relu')(y)
y=Dropout(0.1)(y)
outputs=Dense(activation='softmax',units=2)(y)


In [ ]:
model=Model(inputs=inputs,outputs=outputs)


In [ ]:
model.summary()

In [ ]:
plot_model(model,dpi=60,show_shapes=True)

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer=Adam(learning_rate=0.001),metrics=['accuracy','auc'])


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

classes = np.unique(y_train)
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y = np.argmax(y_train, axis=1)

)

class_weights = dict(zip(classes, class_weights))



In [ ]:

model.fit(X_train,y_train,epochs=20,batch_size=520,class_weight=class_weights)

In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    depth=8,
    learning_rate=0.05,
    iterations=2000,
    loss_function='Logloss',
    eval_metric='AUC',
    auto_class_weights='Balanced'
)
model.fit(X_train, np.argmax(y_train,axis=1),verbose=0);

In [ ]:
y=model.predict(X_test)

In [ ]:
y=np.argmax(y,axis=1)

In [ ]:
df=pd.read_csv('/kaggle/input/playground-series-s5e11/sample_submission.csv')

In [ ]:
print(y)

In [ ]:
df['loan_paid_back']=y

In [ ]:
df

In [ ]:
df.to_csv('eighth_submission.csv',index=False)